In [1]:
import pulp
import math
import folium
from IPython.display import IFrame

In [2]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers

    # Converting lat and long from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine Formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

def create_distance_matrix(locations, start_lat, start_lon):
    num_locations = len(locations)
    distance_matrix = [[0] * num_locations for _ in range(num_locations)]

    for i in range(num_locations):
        for j in range(num_locations):
            # Update the distance matrix with the new start location
            distance_matrix[i][j] = haversine(start_lat, start_lon, locations[i]['lat'], locations[i]['lon']) + \
                                    haversine(locations[i]['lat'], locations[i]['lon'],
                                              locations[j]['lat'], locations[j]['lon'])

    return distance_matrix

# Updated start location for Paris Charles de Gaulle Airport
start_lat = 49.0079
start_lon = 2.5487

coffee_shops = [
    {'name': 'Substance Cafe', 'lat': 48.8662, 'lon': 2.3492},
    {'name': 'The Beans on Fire', 'lat': 48.8855, 'lon': 2.3382},
    {'name': 'Dreamin Man', 'lat': 48.8650, 'lon': 2.3664},
    {'name': 'Cafe Nuance', 'lat': 48.8688, 'lon': 2.3312},
    {'name': 'Kawa', 'lat': 48.8630, 'lon': 2.3606},
    {'name': 'Cayo Coffee Roasters', 'lat': 48.8305, 'lon': 2.3790},
    {'name': 'Motors Coffee', 'lat': 48.8591, 'lon': 2.3469},
    {'name': 'KB_Coffeeshop', 'lat': 48.8809, 'lon': 2.3408},
    {'name': 'Telescope Cafe', 'lat': 48.8659, 'lon': 2.3362},
    {'name': 'Noir Coffee Shop', 'lat': 48.8610, 'lon': 2.3225}
]

distance_matrix = create_distance_matrix(coffee_shops, start_lat, start_lon)

In [3]:
n = len(distance_matrix)
model = pulp.LpProblem("Coffee_Shop_Tour", pulp.LpMinimize)

x = pulp.LpVariable.dicts("x", [(i, j) for i in range(n) for j in range(n)], cat="Binary")

# Objective function
model += pulp.lpSum(distance_matrix[i][j] * x[(i, j)] for i in range(n) for j in range(n))

# Constraints
for i in range(n):
    model += pulp.lpSum(x[(i, j)] for j in range(n)) == 1  # Visit each location once
    model += pulp.lpSum(x[(j, i)] for j in range(n)) == 1  # Leave each location once

# Solve the model
model.solve()

optimal_route = [i for i in range(n) for j in range(n) if pulp.value(x[(i, j)]) == 1]
print("Optimal Route:", optimal_route)

optimal_order = [j + 1 for j in optimal_route]
print("Optimal Order of Locations:", optimal_order)

optimal_route_coordinates = [[coffee_shops[i]['lat'], coffee_shops[i]['lon']] for i in optimal_route]
optimal_route_coordinates.append(optimal_route_coordinates[0])  # Closing the loop

# Find the coordinates of the first coffee shop in the optimal order
first_shop_coordinates = [coffee_shops[optimal_route[0]]['lat'], coffee_shops[optimal_route[0]]['lon']]

# Create a map centered at the first coffee shop in the optimal order
mymap = folium.Map(location=first_shop_coordinates, zoom_start=13)

# Add markers for coffee shops with numbers
for i, shop in enumerate(coffee_shops):
    folium.Marker([shop['lat'], shop['lon']], popup=f"{i + 1}. {shop['name']}").add_to(mymap)

# Add a polyline for the optimal route
folium.PolyLine(optimal_route_coordinates, color='blue', weight=2.5, opacity=1).add_to(mymap)

# Add numbers to the optimal route
for i, coord in enumerate(optimal_route_coordinates[:-1]):
    folium.Marker(coord, icon=folium.DivIcon(html=f"<div style='font-size: 20; color: black;'>{i + 1}</div>")).add_to(mymap)

# Save the map as an HTML file
html_file_path = "optimal_route_map_with_numbers.html"
mymap.save(html_file_path)

# Display the map within Kaggle notebook using IFrame
IFrame(html_file_path, width=800, height=600)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/0728f62989e947388e5567c0dca87380-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/0728f62989e947388e5567c0dca87380-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 25 COLUMNS
At line 526 RHS
At line 547 BOUNDS
At line 648 ENDATA
Problem MODEL has 20 rows, 100 columns and 200 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 217.642 - 0.00 seconds
Cgl0004I processed model has 20 rows, 100 columns (100 integer (100 of which binary)) and 200 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 217.642
Cbc0038I Before mini branch and bound, 100 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did not improve solution (0.01 seconds)
Cbc0038